### **Script base: spazzata in frequenza**

Il notebook mostra come fare una scansione in frequenza per la misura del diagramma di Bode. Lo script si basa 
non su una FFT ma semmai su un fit con un coseno.

Parametri chiave

* `nper` imposta il numero di periodi misurati nell'acquisizione... dato il metodo di stima (fit), ne basta uno
* `nf` imposta il numero di frequenze nello scan
* `npt` imposta il numero dei punti da acquisire
* `f0` imposta la frequenza iniziale
* `f1` imposta la frequenza finale
* `flag_return` => attiva andata e ritorno
    
> **Come è scelto il campionamento?**. Lo script cerca di acquisire `nper` periodi con `npt` punti che, dato un segnale a frequenza `ff` in teoria richiede a
>
> ```python
>        .fc = ff * npt / nper
> ```
>
>tuttavia, $\texttt{Analog Discovery 2}$ può solo campionare a sottomultipli $(100/n)\,{\rm MHz}$. Lo script quindi calcola $n$ con la formula di sopra e lo arrotonda *verso l'alto*, calcolando poi quale sia l'esatto numero di punto per coprire il numero di periodi richiesto. L'arrotondamento verso l'alto è stato scelto in modo che il numero di punti finale non superi mai quello impostato nei parametri.

> **Attenzione**. La qualità dello studio declina rapidamente sopra i $100\,{\rm kHz}$ ed è certamente sconsigliato andare sopra $1\,{\rm MHz}$. Il motivo più probabile per le discrepanze che si sviluppano è legato al cross-talk fra i cavi del *bundle* fornito, che non è particolarmente adatto alle alte frequenze. 

In [4]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as so
import math
    
# ==========================================================================
#  Parametri dello script
nper = 2            # numero periodi usati per la stima   
npt = 8192          # numero MASSIMO di punti acquisiti
nf = 200            # numero di frequenze nello sweep da f0 a f1   
f0 = 0.5        
f1 = 100e3      
flag_return = False  # A/R? o solo andata?
flag_show = True    # Visualizza i segnali nel tempo? (è uguale...)
# vettore delle frequenze
fv = np.logspace(np.log10(f0), np.log10(f1), nf) 

# ==========================================================================
#  Configurazione base AD2 (più parametri impostati dopo)
ad2 = tdwf.AD2()
ad2.vdd = 5
ad2.vss = -5
ad2.power(True)

wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.ampl = 4.5
wavegen.w1.func = tdwf.funcSine 
wavegen.w1.start()
scope = tdwf.Scope(ad2.hdwf)
scope.ch1.rng = 50
scope.ch2.rng = 50

# ==========================================================================
ffv1 = []
AAm1 = []
pphim1 = []

# ==========================================================================
#   Esecuzione...
acquisitions=20
# [1] Crea plot
for i in range(acquisitions):
    # [2] Preparazione dataset
    flag_first = True                # primo plot diverso dai seguenti (update)
    Am = np.full((nf, 2), np.nan)    # pre-allocazione dati (nan non sono plottati!) 
    phim = np.full((nf, 2), np.nan)  # pre-allocazione dati
    # [3] Ciclo misura
    for ar in range(2 if flag_return else 1): # Ciclo A/R
        for ii in range(len(fv)):  # Ciclo frequenze
            # [3a] calcolo frequenza
            if ar==0: # caso A(ndata)
                findex = ii
            else:     # caso R(itorno)
                findex = len(fv)-ii-1
            # Frequenza attuale
            ff = fv[findex]
            # [3b] stima parametri di sampling
            #
            #  COSA VOGLIO: misurare nper con al massimo npt punti acquisizione
            #  DOMANDA: quale è la MASSIMA frequenza di sampling che posso usare?
            #
            #  NOTARE: solo 100MSa/s intero (qui df) è una frequenza ammessa.
            #
            #  SE voglio misurare nper periodi a ff devo misura per un tempo TT = nper/ff
            #  SE misuro ad un rate fs, mi servono npt = fs*nper/ff punti di acquisizione
            #
            #  voglio che fs*nper/ff sia il più alto possibile ma al massimo uguale a npt 
            #  (altrimenti no buco il buffer...), per ottenere questo sceglo un df intero 
            #  in modo che fs = 100MHz/df soddisfi la relazione sopra
            #
            #  => df = celing(100MHz*nper/(npt*ff)) 
            #
            df = math.ceil(100e6*nper/(npt*ff))
            scope.fs = 100e6/df
            scope.npt = int(scope.fs*nper/ff)
            #  Ribadiamo il trigger... 
            scope.trig(True, hist = 0.01)
            wavegen.w1.freq = ff 
            # [3c] Campionamento e analisi risultati
            scope.sample()
            
            A1 = (np.mean(scope.ch1.vals[max(np.argmax(scope.ch1.vals) - 3, 0)
                    : min(np.argmax(scope.ch1.vals) + 4, len(scope.ch1.vals))])
                    - np.mean(scope.ch1.vals[max(np.argmin(scope.ch1.vals) - 3, 0)
                    : min(np.argmin(scope.ch1.vals) + 4, len(scope.ch1.vals))]))

            A2 = (np.mean(scope.ch2.vals[max(np.argmax(scope.ch2.vals) - 3, 0)
                    : min(np.argmax(scope.ch2.vals) + 4, len(scope.ch2.vals))])
                    - np.mean(scope.ch2.vals[max(np.argmin(scope.ch2.vals) - 3, 0)
                    : min(np.argmin(scope.ch2.vals) + 4, len(scope.ch2.vals))]))

            Am[findex, ar] = A2 / A1
      
    ffv1.append(ff)
    AAm1.append(Am)
    print(f"Gathering data: {(i+1)*100/acquisitions}%")
print("DONE!")

# Converto le liste in array numpy
AAm1 = np.array(AAm1)       # shape (nmisure, nf, 2)

# Medie e deviazioni standard per ogni frequenza
AAm   = np.mean(AAm1, axis=0)   # shape (nf, 2)
sAAm  = np.std(AAm1, axis=0)    # shape (nf, 2)

# Frequenze: basta salvarle una volta, non dentro il loop
ffv = fv


# --------------------------------------- Plot dati

plt.figure(1)
plt.errorbar(ffv, AAm[:,0],sAAm[:,0], label="Gain", fmt='o')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which="both", ls="dotted")
plt.show()

data = np.column_stack((fv, AAm[:,0], sAAm[:,0]))
np.savetxt("AD8031_Gain1e3_maxmethod.txt", data, header="f  Gain  sGain")


# ---------------------------------------
ad2.close()



Dispositivo #1 [SN:210321B5DD07, hdwf=1] connesso!
Configurazione #1
Gathering data: 5.0%
Gathering data: 10.0%
Gathering data: 15.0%
Gathering data: 20.0%
Gathering data: 25.0%
Gathering data: 30.0%
Gathering data: 35.0%
Gathering data: 40.0%
Gathering data: 45.0%
Gathering data: 50.0%
Gathering data: 55.0%
Gathering data: 60.0%
Gathering data: 65.0%
Gathering data: 70.0%
Gathering data: 75.0%
Gathering data: 80.0%
Gathering data: 85.0%
Gathering data: 90.0%
Gathering data: 95.0%
Gathering data: 100.0%
DONE!
Dispositivo disconnesso.


# Salvataggio dei dati...

Ovviamente si potrebbe mettere in un unico script, ma è sano poter cambiare qualcosa qui senza dover runnare da zero la misura...

In [8]:
data = np.column_stack([fv,Am, phim])
np.savetxt("Output.txt", data)